## Importing libraries

In [14]:
import pandas as pd
import numpy as np

## Loading the data set

The data set is assigned to a Pandas data frame.

In [25]:
df = pd.read_csv("data/spect.csv", sep=',', index_col=False, decimal=',')

# Types of attributes and null values

Description of the dataset

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   OVERALL_DIAGNOSIS  267 non-null    int64
 1   F1                 267 non-null    int64
 2   F2                 267 non-null    int64
 3   F3                 267 non-null    int64
 4   F4                 267 non-null    int64
 5   F5                 267 non-null    int64
 6   F6                 267 non-null    int64
 7   F7                 267 non-null    int64
 8   F8                 267 non-null    int64
 9   F9                 267 non-null    int64
 10  F10                267 non-null    int64
 11  F11                267 non-null    int64
 12  F12                267 non-null    int64
 13  F13                267 non-null    int64
 14  F14                267 non-null    int64
 15  F15                267 non-null    int64
 16  F16                267 non-null    int64
 17  F17             

Count of null values, for each attribute

In [28]:
df.isnull().sum()

OVERALL_DIAGNOSIS    0
F1                   0
F2                   0
F3                   0
F4                   0
F5                   0
F6                   0
F7                   0
F8                   0
F9                   0
F10                  0
F11                  0
F12                  0
F13                  0
F14                  0
F15                  0
F16                  0
F17                  0
F18                  0
F19                  0
F20                  0
F21                  0
F22                  0
dtype: int64

As you can see, we don't have any missing
values in the data sets.

# Remove noise and outliers


In [9]:
from scipy.stats import zscore
z_scores = zscore(df)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)

if len(filtered_entries)!=len(df):
    df[filtered_entries].to_csv(r'data/spect.csv', index=True, header=True)

df.describe()

,OVERALL_DIAGNOSIS,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
count,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,...,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000
mean,0.794007,0.445693,0.247191,0.393258,0.284644,0.404494,0.235955,0.284644,0.426966,0.310861,...,0.494382,0.303371,0.176030,0.310861,0.142322,0.131086,0.247191,0.322097,0.363296,0.411985
std,0.405185,0.497975,0.432189,0.489391,0.452092,0.491716,0.425392,0.452092,0.495566,0.463715,...,0.500907,0.460577,0.381561,0.463715,0.350036,0.338128,0.432189,0.468158,0.481852,0.493117
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
